<div style="display: flex; background-color: RGB(395,214,220);" >
  <h1 style="margin: auto; padding: 30px; text-align: center;">
    <span>Organisation nationale de lutte contre le
faux-monnayage (ONCFM)</span><br>
    <span>Détecter des faux billets avec Python</span>
  </h1>
</div>

# **OBJECTIFS DE CE NOTEBOOK**
#### - Importer le modèle de machine learning retenu.
#### - L'appliquer sur un nouveau jeu de données.

## Sommaire

- [Etape 1 - Importation des librairies et chargement des données](#etape-1)
  - [1.1 Importation des librairies](#1-1)
  - [1.2 Importation des données](#1-2)
- [Etape 2 - Analyse exploratoire des données](#etape-2)
  - [2.1 Analyse rapide et ajustements](#2-1)
  - [2.2 Régression linéaire variable "Marge_basse"](#2-2)
    - [2.2.1 Analyse des corrélations](#2-2-1)
    - [2.2.2 Test et entrainement du modèle](#2-2-2)
    - [2.2.3 Tests de validité et de fiabilité du modèle](#2-2-3)

<div style="background-color: RGB(395,214,205);">
    <h2 id="etape-1" style="margin: auto; padding: 20px;">Etape 1 - Importation des librairies et chargement des données</h2>
</div>

In [132]:
# Importation des librairies
import pandas as pd
import joblib

In [133]:
# Importation du fichier billets_production.csv
chemin_nouveau_fichier = '/Users/adrianaguilera/Desktop/P12/Inputs/billets_production.csv'
nv_billets = pd.read_csv(chemin_nouveau_fichier)

In [134]:
# Importation du modèle
modele = joblib.load('modele_RF.joblib')

In [135]:
# Affichage des données
nv_billets

,diagonal,height_left,height_right,margin_low,margin_up,length,id
0,171.76,104.01,103.54,5.21,3.30,111.42,A_1
1,171.87,104.17,104.13,6.00,3.31,112.09,A_2
2,172.00,104.58,104.29,4.99,3.39,111.57,A_3
3,172.49,104.55,104.34,4.44,3.03,113.20,A_4
4,171.65,103.63,103.56,3.77,3.16,113.33,A_5


<div style="background-color: RGB(395,214,205);">
    <h2 id="etape-2" style="margin: auto; padding: 20px;">Etape 2 - Application du modèle de prédiction</h2>
</div>

In [136]:
# Renommer les variables
nv_billets = nv_billets.rename(columns={
    "is_genuine": "Authenticite",
    "diagonal": "Diagonale",
    "height_left": "Hauteur_gauche",
    "height_right": "Hauteur_droite",
    "margin_low": "Marge_basse",
    "margin_up": "Marge_haute",
    "length": "Longueur"
})
# Supprimer la colonne 'id' si elle est présente
nv_billets_clean = nv_billets.drop(columns=['id'], errors='ignore')
# Appliquer la prédiction sur les données corrigées
y_pred = modele.predict(nv_billets_clean)

In [137]:
# Ajouter la colonne 'Authenticite' à partir des prédictions
nv_billets_clean['Authenticite'] = y_pred
# Ajouter la colonne 'id' à partir du fichier d'origine
nv_billets_clean['id'] = nv_billets['id']
# Liste des colonnes dans l'ordre souhaité
ordre_colonne = ['id', 'Authenticite', 'Diagonale', 'Hauteur_gauche', 'Hauteur_droite',
                  'Marge_basse', 'Marge_haute', 'Longueur']
# Réorganiser les colonnes
nv_billets_clean = nv_billets_clean[ordre_colonne]
# Affichage des données
nv_billets_clean

,id,Authenticite,Diagonale,Hauteur_gauche,Hauteur_droite,Marge_basse,Marge_haute,Longueur
0,A_1,0,171.76,104.01,103.54,5.21,3.30,111.42
1,A_2,0,171.87,104.17,104.13,6.00,3.31,112.09
2,A_3,0,172.00,104.58,104.29,4.99,3.39,111.57
3,A_4,1,172.49,104.55,104.34,4.44,3.03,113.20
4,A_5,1,171.65,103.63,103.56,3.77,3.16,113.33


<div style="background-color: RGB(395,214,205);">
    <h2 id="etape-3" style="margin: auto; padding: 20px;">Etape 3 - Affichage des résultats</h2>
</div>

In [138]:
# Compter le nombre de billets
nb_billets = len(nv_billets_clean)
# Compter le nombre de billets authentiques
nb_billets_authentiques = len(nv_billets_clean[nv_billets_clean['Authenticite'] == 1])
# Compter le nombre de billets contrefaits
nb_billets_contrefaits = len(nv_billets_clean[nv_billets_clean['Authenticite'] == 0])
# Afficher le nombre de billets authentiques et contrefaits
print(f"Nombre total de billets : {nb_billets}")
print(f"Nombre de billets authentiques : {nb_billets_authentiques}")
print(f"Nombre de billets contrefaits : {nb_billets_contrefaits}")
# Pourcentage de billets authentiques 
pourcentage_authentiques = (nb_billets_authentiques / nb_billets) * 100
print(f"Pourcentage de billets authentiques : {pourcentage_authentiques:.2f}%")


Nombre total de billets : 5
Nombre de billets authentiques : 2
Nombre de billets contrefaits : 3
Pourcentage de billets authentiques : 40.00%


In [139]:
# Exporter les données
#nv_billets_clean.to_csv('resultats_billets.csv', index=False)
#print("Fichier 'resultats_billets.csv' exporté avec succès.")